In [1]:
pip install pytorchtools


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Thư viện hệ thống
import os
import time
from collections import defaultdict

# Xử lý XML và ảnh
import xml.etree.ElementTree as ET
from PIL import Image
import cv2
import numpy as np

# Torch và torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import torchvision.transforms as T
import torchvision.models as models
from torchvision.ops import (
    box_convert, generalized_box_iou, generalized_box_iou_loss
)

# Thư viện augmentation và chuyển đổi tensor
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Vẽ ảnh
import matplotlib.pyplot as plt
import matplotlib.patches as patches



In [6]:


class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f"Validation loss decreased ({self.val_loss_min:.6f} → {val_loss:.6f}). Saving model ...")
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


In [7]:
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    objects = []
    for obj in root.findall('object'):
        label = obj.find('name').text
        bbox = obj.find('bndbox')
        box = {
            'label': label,
            'bbox': {
                'xmin': int(bbox.find('xmin').text),
                'ymin': int(bbox.find('ymin').text),
                'xmax': int(bbox.find('xmax').text),
                'ymax': int(bbox.find('ymax').text)
            }
        }
        objects.append(box)
    return objects

def plot_image_with_boxes(image_path, annotation_path, save_path = None):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    objects = parse_annotation(annotation_path)

    for obj in objects:
        label = obj['label']
        bbox = obj['bbox']
        cv2.rectangle(image, 
                      (bbox['xmin'], bbox['ymin']), 
                      (bbox['xmax'], bbox['ymax']), 
                      color=(255, 0, 0), thickness=2)
        cv2.putText(image, label, 
                    (bbox['xmin'], bbox['ymin'] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, 
                    (0, 255, 0), 2)

    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    
    if save_path:
        image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        cv2.imwrite(save_path, image_bgr)
        print(f"✅ Đã lưu ảnh có bounding box tại: {save_path}")

# Đường dẫn tuyệt đối đến file ảnh và file annotation
image_path = '/kaggle/input/helo123/images/Abyssinian_1.jpg'
annotation_path = '/kaggle/input/helo123/annotations/xmls/Abyssinian_1.xml'

# Gọi hàm để vẽ ảnh với bounding box
plot_image_with_boxes(image_path, annotation_path, save_path = '/kaggle/working/Abyssinian_1_bbox.jpg' )

[ WARN:0@58.066] global loadsave.cpp:241 findDecoder imread_('/kaggle/input/helo123/images/Abyssinian_1.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.10.0) /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_81ng91bl78/croot/opencv-suite_1738943359148/work/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [8]:
import os
import shutil

# Đường dẫn tới ảnh và annotation
image_dir = '/kaggle/input/helo123/images'
xml_dir = '/kaggle/input/helo123/annotations/xmls'
backup_dir = '/kaggle/working//no_annotation_images'

# Tạo thư mục nếu chưa tồn tại
os.makedirs(backup_dir, exist_ok=True)

# Lấy danh sách tên file (không kèm đuôi)
image_ids = set(f.replace('.jpg', '') for f in os.listdir(image_dir) if f.endswith('.jpg'))
xml_ids = set(f.replace('.xml', '') for f in os.listdir(xml_dir) if f.endswith('.xml'))

# Tìm ảnh không có annotation
no_xml_ids = image_ids - xml_ids
print(f"📦 Số ảnh không có annotation: {len(no_xml_ids)}")

# Di chuyển từng ảnh vào thư mục backup
for img_id in no_xml_ids:
    src = os.path.join(image_dir, img_id + '.jpg')
    dst = os.path.join(backup_dir, img_id + '.jpg')
    if os.path.exists(src):
        shutil.move(src, dst)

print(f"✅ Đã di chuyển {len(no_xml_ids)} ảnh sang thư mục: {backup_dir}")

OSError: [Errno 30] Read-only file system: '/kaggle'

In [9]:
import random
import shutil

def random_split(ids, train_ratio, val_ratio, test_ratio):
    assert train_ratio + val_ratio + test_ratio == 1.0, "Tỉ lệ chia không hợp lệ!"
    
    random.shuffle(ids)
    total_size = len(ids)
    
    train_size = int(total_size * train_ratio)
    val_size = int(total_size * val_ratio)
    test_size = total_size - train_size - val_size

    train_ids = ids[:train_size]
    val_ids = ids[train_size:train_size + val_size]
    test_ids = ids[train_size + val_size:]

    return train_ids, val_ids, test_ids


In [10]:
class PetDataset(Dataset):
    def __init__(self, image_dir, xml_dir, image_ids, transform=None):
        self.image_dir = image_dir
        self.xml_dir = xml_dir
        self.image_ids = image_ids
        self.transform = transform

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_name = self.image_ids[idx]
        img_path = os.path.join(self.image_dir, img_name)
        xml_name = img_name.replace('.jpg', '.xml')
        xml_path = os.path.join(self.xml_dir, xml_name)

        img = Image.open(img_path).convert('RGB')

        tree = ET.parse(xml_path)
        root = tree.getroot()
        obj = root.find('object')
        label = 0 if obj.find('name').text.lower() == 'cat' else 1

        bndbox = obj.find('bndbox')
        box = [
            float(bndbox.find('xmin').text),
            float(bndbox.find('ymin').text),
            float(bndbox.find('xmax').text),
            float(bndbox.find('ymax').text)
        ]

        if self.transform:
            img_np = np.array(img)
            transformed = self.transform(image=img_np, bboxes=[box], labels=[label])
            img = transformed['image']
            box = transformed['bboxes'][0]
            label = transformed['labels'][0]

        return img, torch.tensor(label, dtype=torch.long), torch.tensor(box, dtype=torch.float32)


In [11]:
train_transform = A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3), 
    A.ShiftScaleRotate(shift_limit=0.02, scale_limit=0.1, rotate_limit=10, p=0.3, border_mode=0),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))


val_test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),  # Cần chuẩn hóa ảnh trước khi vào model pretrain
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
# Đường dẫn dữ liệu
image_dir = '/kaggle/input/helo123/images'
xml_dir = '/kaggle/input/helo123/annotations/xmls'

# Lấy danh sách file ảnh
image_ids = sorted([f for f in os.listdir(image_dir) if f.endswith('.jpg')])

# Chia train, val, test
train_ids, val_ids, test_ids = random_split(image_ids, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1)

# Khởi tạo Dataset
train_dataset = PetDataset(image_dir, xml_dir, train_ids, transform=train_transform)
val_dataset   = PetDataset(image_dir, xml_dir, val_ids, transform=val_test_transform)
test_dataset  = PetDataset(image_dir, xml_dir, test_ids, transform=val_test_transform)

# Tạo DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Kiểm tra
for images, labels, bboxes in train_loader:
    print("Train batch - Images:", images.shape, "Labels:", labels.shape, "BBoxes:", bboxes.shape)
    break

/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/albumentations/core/validation.py:111: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/helo123/images'

In [12]:
class ResNetDetector(nn.Module):
    def __init__(self, num_classes=2, freeze_stem=True):
        super(ResNetDetector, self).__init__()
        resnet = models.resnet50(pretrained=True)

        # Phần stem: conv1 → bn1 → relu → maxpool
        self.stem = nn.Sequential(
            resnet.conv1,
            resnet.bn1,
            resnet.relu,
            resnet.maxpool,
        )
        
        # Luôn train layer1–layer4
        self.layer1 = resnet.layer1
        self.layer2 = resnet.layer2
        self.layer3 = resnet.layer3
        self.layer4 = resnet.layer4

        if freeze_stem:
            for param in self.stem.parameters():
                param.requires_grad = False

        # Pooling & heads
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.classifier = nn.Linear(2048, num_classes)
        self.bbox_regressor = nn.Linear(2048, 4)

    def forward(self, x):
        x = self.stem(x)           # [B,  64, H/4, W/4]
        x = self.layer1(x)         # [B, 256, H/4, W/4]
        x = self.layer2(x)         # [B, 512, H/8, W/8]
        x = self.layer3(x)         # [B,1024, H/16,W/16]
        x = self.layer4(x)         # [B,2048, H/32,W/32]
        x = self.avgpool(x)        # [B,2048, 1, 1]
        flat = self.flatten(x)     # [B,2048]

        cls_logits = self.classifier(flat)
        bbox_pred  = self.bbox_regressor(flat)
        return bbox_pred, cls_logits


In [13]:
import torch
import torch.nn as nn
from torchvision.ops import box_convert, generalized_box_iou

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNetDetector(num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
early_stopping = EarlyStopping(patience=10, verbose=True, path='/kaggle/working/resnet100_earlystop_best.pth') 
num_epochs = 100

# Loss functions
criterion_cls = nn.BCEWithLogitsLoss(reduction='mean')
criterion_bbox = nn.SmoothL1Loss(reduction='mean')

def compute_bounding_box_loss(pred_cls, pred_boxes, gt_labels, gt_boxes):
    pred_boxes_xyxy = box_convert(pred_boxes, in_fmt='cxcywh', out_fmt='xyxy')
    gt_boxes_xyxy = box_convert(gt_boxes, in_fmt='cxcywh', out_fmt='xyxy')

    ious = generalized_box_iou(pred_boxes_xyxy, gt_boxes_xyxy).diag()

    batch_size = pred_cls.size(0)
    num_classes = pred_cls.size(1)
    gt_onehot = torch.zeros((batch_size, num_classes), device=pred_cls.device)
    gt_onehot.scatter_(1, gt_labels.unsqueeze(1), 1.0)

    soft_labels = gt_onehot * ious.unsqueeze(1)
    cls_loss = criterion_cls(pred_cls, soft_labels)
    bbox_loss = criterion_bbox(pred_boxes, gt_boxes)

    return cls_loss + bbox_loss

# # --- Training loop ---
# best_val_loss = float('inf')
# best_model_path = 'resnet50_100epoch.pth'
# train_losses = []
# val_losses = []

# for epoch in range(num_epochs):
#     # --- Training ---
#     model.train()
#     running_loss = 0.0

#     for images, gt_labels, gt_boxes in train_loader:
#         images = images.to(device)
#         gt_labels = gt_labels.to(device)
#         gt_boxes = gt_boxes.to(device)

#         pred_boxes, pred_cls = model(images)
#         loss = compute_bounding_box_loss(pred_cls, pred_boxes, gt_labels, gt_boxes)

#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item() * images.size(0)

#     epoch_loss = running_loss / len(train_loader.dataset)
#     train_losses.append(epoch_loss)

#     # --- Validation ---
#     model.eval()
#     val_running_loss = 0.0
#     with torch.no_grad():
#         for images, gt_labels, gt_boxes in val_loader:
#             images = images.to(device)
#             gt_labels = gt_labels.to(device)
#             gt_boxes = gt_boxes.to(device)

#             pred_boxes, pred_cls = model(images)
#             val_loss = compute_bounding_box_loss(pred_cls, pred_boxes, gt_labels, gt_boxes)

#             val_running_loss += val_loss.item() * images.size(0)

#     val_epoch_loss = val_running_loss / len(val_loader.dataset)
#     val_losses.append(val_epoch_loss)

#  # --- Print log ---
#     print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_epoch_loss:.4f}")

#     # --- EarlyStopping check ---
#     early_stopping(val_epoch_loss, model)

#     # --- Save losses ---
#     torch.save({
#         'train_losses': train_losses,
#         'val_losses': val_losses
#     }, 'losses.pt')

#     if early_stopping.early_stop:
#         print("=> Early stopping triggered.")
#         break

/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /Users/minhkha/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:32<00:00, 3.17MB/s]


In [14]:
from sklearn.metrics import classification_report
from collections import defaultdict
import torch

def compute_iou(box1, box2):
    """Tính IoU giữa hai hộp giới hạn."""
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    iou = inter_area / union_area if union_area > 0 else 0
    return iou

def evaluate_model(model, dataloader, compute_bounding_box_loss, generalized_box_iou, iou_threshold=0.5, device="cuda"):
    model.eval()
    total_loss = 0
    total_samples = 0
    correct_localization = 0
    correct_both = 0
    total_iou = 0
    total_giou = 0
    all_precisions = []
    all_recalls = []

    y_true = []
    y_pred = []

    with torch.no_grad():
        for images, gt_labels, gt_boxes in dataloader:
            images = images.to(device)
            gt_labels = gt_labels.to(device)
            gt_boxes = gt_boxes.to(device)

            pred_boxes, pred_cls = model(images)
            loss = compute_bounding_box_loss(pred_cls, pred_boxes, gt_labels, gt_boxes)
            total_loss += loss.item() * images.size(0)

            pred_labels = pred_cls.argmax(dim=1)

            for pred_box, pred_label, gt_box, gt_label in zip(pred_boxes, pred_labels, gt_boxes, gt_labels):
                pred_box = pred_box.cpu()
                gt_box = gt_box.cpu()
                pred_label = pred_label.cpu()
                gt_label = gt_label.cpu()

                iou = compute_iou(pred_box, gt_box)

                total_iou += iou

                if iou >= iou_threshold:
                    correct_localization += 1
                if pred_label == gt_label and iou >= iou_threshold:
                    correct_both += 1

                tp = int(pred_label == gt_label and iou >= iou_threshold)
                fp = int(pred_label != gt_label)
                fn = int(iou < iou_threshold)

                precision = tp / (tp + fp + 1e-6)
                recall = tp / (tp + fn + 1e-6)

                all_precisions.append(precision)
                all_recalls.append(recall)

                total_samples += 1

                y_true.append(gt_label.item())
                y_pred.append(pred_label.item())

    avg_loss = total_loss / total_samples
    localization_accuracy = correct_localization / total_samples
    combined_accuracy = correct_both / total_samples
    mean_iou = total_iou / total_samples
    avg_precision = sum(all_precisions) / len(all_precisions)
    avg_recall = sum(all_recalls) / len(all_recalls)

    # Tạo báo cáo phân loại chi tiết
    report = classification_report(y_true, y_pred, digits=4, output_dict=True, zero_division=0)

    # In kết quả
    print(f"Loss: {avg_loss:.4f}")
    print(f"- Độ chính xác định vị (IoU >= {iou_threshold}): {localization_accuracy:.4f}")
    print(f"- Độ chính xác kết hợp: {combined_accuracy:.4f}")
    print(f"- Trung bình IoU: {mean_iou:.4f}")
    print("\n Classification Report:")
    print(classification_report(y_true, y_pred, digits=4, zero_division=0))

    return {
        "loss": avg_loss,
        "loc_acc": localization_accuracy,
        "combined_acc": combined_accuracy,
        "mean_iou": mean_iou,
        "classification_report": report,
    }


In [15]:
# Hàm trực quan hóa dự đoán
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ResNetDetector(num_classes=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
early_stopping = EarlyStopping(patience=10, verbose=True, path='/kaggle/working/resnet100_earlystop_best.pth') 

# Loss functions
criterion_cls = nn.BCEWithLogitsLoss(reduction='mean')
criterion_bbox = nn.SmoothL1Loss(reduction='mean')

def visualize_predictions(model, dataset, idx=0, device="cuda", mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """Hiển thị ảnh với hộp dự đoán và hộp thực tế."""
    model.eval()
    img, gt_label, gt_box = dataset[idx]
    img_tensor = img.unsqueeze(0).to(device)

    with torch.no_grad():
        pred_box, pred_cls = model(img_tensor)
        pred_label = pred_cls[0].argmax().item()
        pred_box = pred_box[0].cpu()

    # Đảo ngược chuẩn hóa ảnh
    img_np = img.permute(1, 2, 0).numpy()
    img_np = img_np * np.array(std) + np.array(mean)
    img_np = np.clip(img_np, 0, 1)

    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    ax.imshow(img_np)

    # Vẽ hộp thực tế (ground-truth)
    rect = patches.Rectangle((gt_box[0], gt_box[1]), gt_box[2]-gt_box[0], gt_box[3]-gt_box[1],
                             linewidth=2, edgecolor='green', facecolor='none')
    ax.add_patch(rect)
    ax.text(gt_box[0], gt_box[1] - 4, f"GT: {gt_label.item()}", color='green')

    # Vẽ hộp dự đoán
    rect = patches.Rectangle((pred_box[0], pred_box[1]), pred_box[2]-pred_box[0], pred_box[3]-pred_box[1],
                             linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)
    ax.text(pred_box[0], pred_box[1] + 4, f"Pred: {pred_label}", color='red')

    plt.axis("off")
    plt.tight_layout()
    plt.show()



# # Load mô hình tốt nhất trước khi đánh giá
# model.load_state_dict(torch.load('/kaggle/input/hehe123/resnet100_earlystop_best.pth'))
# model.eval()

# # Đánh giá và trực quan hóa trên tập test
# print("\nĐánh giá trên tập test:")
# test_loss= evaluate_model(
#     model, test_loader,  compute_bounding_box_loss,generalized_box_iou=generalized_box_iou, iou_threshold=0.5, device=device
# )

# print("Trực quan hóa một số mẫu từ tập test:")
# for idx in range(5):
#     if idx < len(test_dataset):
#         print(f"Mẫu test {idx}:")
#         visualize_predictions(model, test_dataset, idx=idx, device=device)

/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
import matplotlib.pyplot as plt

# Vẽ đồ thị loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Train Loss', marker='o')
plt.plot(val_losses, label='Validation Loss', marker='s')
plt.title('Training vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'train_losses' is not defined

<Figure size 1000x600 with 0 Axes>

In [20]:
from torchvision import transforms

In [31]:
# ======= CẤU HÌNH =======
input_dir = "/Users/minhkha/Downloads/images-2.jpeg"
output_dir = "/Users/minhkha/Desktop/minh_output"
os.makedirs(output_dir, exist_ok=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ======= THÔNG SỐ CHUẨN HÓA (nếu dùng ImageNet) =======
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# ======= LOAD MÔ HÌNH =======
model = ResNetDetector()
model.load_state_dict(torch.load('/Users/minhkha/Downloads/cs231-official.pth', map_location=device))
model.to(device)
model.eval()

# ======= TIỀN XỬ LÝ ẢNH =======
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

# ======= LẶP QUA CÁC ẢNH =======
# for img_file in os.listdir(input_dir):
#     if not img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
#         continue

img_path = input_dir
image = Image.open(img_path).convert("RGB")
image_resized = image.resize((224, 224))  # để vẽ lại

input_tensor = transform(image).unsqueeze(0).to(device)

with torch.no_grad():
        pred_box, pred_cls = model(input_tensor)
        pred_label = pred_cls[0].argmax().item()
        pred_box = pred_box[0].cpu().numpy()  # [x1, y1, x2, y2]

    # VẼ KẾT QUẢ
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.imshow(image_resized)

rect = patches.Rectangle(
        (pred_box[0], pred_box[1]),
        pred_box[2] - pred_box[0],
        pred_box[3] - pred_box[1],
        linewidth=2, edgecolor='red', facecolor='none'
    )
ax.add_patch(rect)
ax.text(pred_box[0], pred_box[1] + 4, f"Pred: {pred_label}", color='green')
ax.axis("off")
img_name = os.path.basename(img_path)
output_path = os.path.join(output_dir, img_name)

plt.tight_layout()
plt.title(f"Pred: {pred_label}")
plt.savefig(output_path, bbox_inches='tight')
plt.close('all')

/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/cv_env/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
